In [ ]:
from pyproj import Proj, transform
import shapefile as sf
import matplotlib.pyplot as plt 
import numpy as np

gron_sf = sf.Reader('FieldShapeFile/Groningen_field')
print('gron_sf:',gron_sf)
print('gron_sf.shapes():',gron_sf.shapes())
gs = gron_sf.shape(0)

gf_bb = gs.bbox
print('gf_bb:',gf_bb)

gf_pts = np.asarray(gs.points)

lat = 53.283266 #picked form google maps just above Schildmeer
lon = 6.813186  #picked form google maps just above Schildmeer

wgs84_proj= Proj('epsg:4326')
nl_proj = Proj('epsg:28992')

x,y = transform(wgs84_proj,nl_proj,lat,lon)
print('x,y = %f,%f' %(x,y))

nlat,nlon = transform(nl_proj,wgs84_proj,x,y)
print('olat,olon = %f,%f' %(lat,lon))
print('nlat,nlon = %f,%f' %(nlat,nlon))

#pbbox_x = np.array([mybbox[0],mybbox[0],mybbox[2],mybbox[2],mybbox[0]])
#pbbox_y = np.array([mybbox[1],mybbox[3],mybbox[3],mybbox[1],mybbox[1]])

#c_loop = np.array([[mybbox[0],mybbox[1]],[mybbox[0],mybbox[3]],
                   #[mybbox[2],mybbox[3]],[mybbox[2],mybbox[1]],
                   #[mybbox[0],mybbox[1]]])
#
#gf_bbox = bb(c_loop)
plt_gfbb_x = np.array([gf_bb[0],gf_bb[2]])
plt_gfbb_y = np.array([gf_bb[1],gf_bb[3]])

xspan = plt_gfbb_x[1] - plt_gfbb_x[0]
yspan = plt_gfbb_y[1] - plt_gfbb_y[0]
llat,llon = transform(nl_proj,wgs84_proj,plt_gfbb_x[0]-xspan,plt_gfbb_y[0]-yspan)
print('llat,llon = %f,%f' %(llat,llon))
ulat,ulon = transform(nl_proj,wgs84_proj,plt_gfbb_x[1]+xspan,plt_gfbb_y[1]+yspan)
print('ulat,ulon = %f,%f' %(ulat,ulon))

lat0 = 0.5*(ulat-llat)
lon0 = 0.5*(ulon-llon)

lx,ly = transform(wgs84_proj,nl_proj,llat,llon)
ux,uy = transform(wgs84_proj,nl_proj,ulat,ulon)

map_gf = np.zeros(gf_pts.shape)
print('gf_pts.shape:',gf_pts.shape)
print('map_gf.shape:',map_gf.shape)

for ic in range(len(gf_pts[:,0])):
    ilat,ilon = transform(nl_proj,wgs84_proj,gf_pts[ic,0],gf_pts[ic,1])
    map_gf[ic,0] = ilat
    map_gf[ic,1] = ilon

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(gf_pts[:,0],gf_pts[:,1],s=1,c='black',zorder=0)
ax.scatter(plt_gfbb_x[0],plt_gfbb_y[0],s=80,c='red',marker='x',zorder=1)
ax.scatter(plt_gfbb_x[1],plt_gfbb_y[1],s=80,c='red',marker='x',zorder=1)
ax.scatter(lx,ly,s=40,c='gray',marker='o',zorder=1)
ax.scatter(ux,uy,s=40,c='blue',marker='o',zorder=1)
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
# setup Lambert Conformal basemap.
m = Basemap(llcrnrlon=llon,llcrnrlat=llat,urcrnrlon=ulon,urcrnrlat=ulat,
            resolution='h',projection='tmerc',lon_0=lon0,lat_0=lat0)
#m = Basemap(width=12000000,height=9000000,projection='lcc',
            #resolution='c',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# draw coastlines.
m.drawcoastlines()
# draw a boundary around the map, fill the background.
# this background will end up being the ocean color, since
# the continents will be drawn on top.
m.drawmapboundary(fill_color='aqua')
# fill continents, set lake color same as ocean color.
m.fillcontinents(color='coral',lake_color='aqua')
m.scatter(map_gf[:,0],map_fg[:,1])
#m.bluemarble()

plt.show()